
FOOTBALL DATA MANAGEMENT


In this file we basically create file which will manage all the api data importation into the discord 
bot that will later on produce the result on discord.

1. Create class
        
2. Associate the commands function
      
3. Connect class with robot file

4. Test class and command function in robot.

In [2]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [3]:
###Different source references

class football:

    def __init__(self):
        """
        Intializing a football that object. This initialization will call the api used 
        to treat the data used in the treatment to produce the diverse commands.
        """
        self.url_topscorers = "https://elenasport-io1.p.rapidapi.com/v2/seasons/4242/topscorers"
        self.url_leagues = "https://elenasport-io1.p.rapidapi.com/v2/leagues"
        self.url_league_id = "https://elenasport-io1.p.rapidapi.com/v2/leagues/263"
        self.url_season = "https://elenasport-io1.p.rapidapi.com/v2/seasons"
        self.url_players = "https://elenasport-io1.p.rapidapi.com/v2/seasons/4242/players"
        self.url_teams = "https://elenasport-io1.p.rapidapi.com/v2/seasons/4242/teams"
        self.url_fixtures_season = "https://elenasport-io1.p.rapidapi.com/v2/seasons/4242/fixtures"
        self.url_upcoming_fixtures = "https://elenasport-io1.p.rapidapi.com/v2/seasons/4242/upcoming"
    
    def collect_data(self,url):
        """
        function to collect all the produced from that API
        input: url (The reference to the desired data to be collected)
        output: data table of collected sets of data.
        """
        querystring = {"page":"1"}
        headers = {
        'x-rapidapi-host': "elenasport-io1.p.rapidapi.com",
        'x-rapidapi-key': "c1858f9ca6mshb6842cc846ee798p1f633cjsnb369850f3cb9"
        }
        i = 2
        response = requests.request("GET", url, headers=headers, params=querystring)
        response_list = response.json()["data"]
        response_df = pd.DataFrame(response_list)
        while response.status_code == 200 and response.json()["pagination"]["hasNextPage"] == True:
            querystring = {"page":str(i)}
            response = requests.request("GET", url, headers=headers, params=querystring)
            try:
                response_df = response_df.append(pd.DataFrame(response.json()["data"]), ignore_index=True)
                #print(response.json()["data"])
                print("page:", i)
                i += 1
            except:
                continue
        return response_df

    def visualize_first_command(self):
        """
        This function permits the visualization of the first command, which produces
        a chart of the 10 best.
        """
        data_topscorers = self.collect_data(self.url_topscorers)
        data_10_topscorers = data_topscorers.sort_values(by=['totalGoals'], ascending = False)[1:11]
        data_10_topscorers
        players = data_10_topscorers['playerName']
        total_goals_without_penalties = data_10_topscorers['totalGoals'] - data_10_topscorers['penaltiesScored']
        penalties_scored = data_10_topscorers['penaltiesScored']
        # Figure Size
        fig, ax = plt.subplots(figsize =(16, 9))
        # Horizontal Bar Plot
        b1 = ax.barh(players, total_goals_without_penalties, color = 'cornflowerblue')
        b2 = ax.barh(players, penalties_scored, left = total_goals_without_penalties, color = 'lightsteelblue')
        # Remove axes splines
        for s in ['top', 'bottom', 'left', 'right']:
            ax.spines[s].set_visible(False)
        # Remove x, y Ticks
        ax.xaxis.set_ticks_position('none')
        ax.yaxis.set_ticks_position('none')
        # Add padding between axes and labels
        ax.xaxis.set_tick_params(pad = 5)
        ax.yaxis.set_tick_params(pad = 10)
        # Add x, y gridlines
        ax.grid(b = True, color ='grey', linestyle ='-.', linewidth = 0.5, alpha = 0.2)
        # Show top values
        ax.invert_yaxis()
        # Add annotation to bars
        for p in ax.patches:
            left, bottom, width, height = p.get_bbox().bounds
            ax.annotate(int(width), xy=(left+width/2, bottom+height/2), 
                        ha='center', va='center', color='white', fontweight ='bold')
        # Add Plot Title
        ax.set_title('10 top scorers in Ligue 1 in the season 2021 - 2022', loc ='left', fontsize=20)

        blue_patch = mpatches.Patch(color='cornflowerblue', label='Total goals without penalties')
        orange_patch = mpatches.Patch(color='lightsteelblue', label='Penalties scored')
        ax.legend(handles=[blue_patch, orange_patch])

        return plt.show()